In [1]:
path_repo_benchmark = "./data/corpus_benchmark"

### renommer 

In [3]:
from pathlib import Path

path_repo_benchmark = Path("./data/corpus_benchmark")

for p in path_repo_benchmark.glob("Copy of FRA*.jpg"):
    new_name = p.name.replace("Copy of ", "", 1)
    target = p.with_name(new_name)
    p.rename(target)

print("done")


done


### créer jsonl de validation

In [4]:
import json
from pathlib import Path

path_repo_benchmark = Path("./data/corpus_benchmark")
out_path = Path("data/corpus_benchmark.jsonl")

items = []
for p in sorted(path_repo_benchmark.glob("FRA*.jpg")):
    items.append({
        "nom_image": p.name,
        "raw_text": "",
        "city": "",
        "hamlet": "",
        "monument": "",
        "other": ""
    })

with out_path.open("w", encoding="utf-8") as f:
    for obj in items:
        f.write(json.dumps(obj, ensure_ascii=False) + "\n")

print("ok ->", out_path, "count:", len(items))


ok -> data/corpus_benchmark.jsonl count: 50


# corpus test

In [5]:
from pathlib import Path

path = Path("data/test20/corpus_test20")

for p in path.iterdir():
    if p.is_file() and p.name.startswith("copy_"):
        new_name = p.name.replace("copy_", "", 1)
        p.rename(p.with_name(new_name))

print("done")


done


In [14]:
import json
from pathlib import Path

path_meta = Path("data/metadata.json")
path_trans = Path("data/test20/transcriptions_coherentes_final.json")
out_path = Path("data/test20/metadata_test20.json")

meta = json.loads(path_meta.read_text(encoding="utf-8"))
trans = json.loads(path_trans.read_text(encoding="utf-8"))

wanted = {t.get("file_name") for t in trans if t.get("file_name")}

file_names = meta.get("File name", [])
keep_idx = [i for i, name in enumerate(file_names) if name in wanted]

filtered = {}
for key, values in meta.items():
    if isinstance(values, list):
        filtered[key] = [values[i] for i in keep_idx]
    else:
        filtered[key] = values

out_path.write_text(json.dumps(filtered, ensure_ascii=False, indent=2), encoding="utf-8")
print("ok ->", out_path, "rows:", len(keep_idx))


ok -> data/test20/metadata_test20.json rows: 20


In [15]:
import json
from pathlib import Path

path_meta = Path("data/test20/metadata_test20.json")
path_trans = Path("data/test20/transcriptions_coherentes_final.json")
out_path = Path("data/test20/metadata_test20_enrichi.json")

meta = json.loads(path_meta.read_text(encoding="utf-8"))
trans = json.loads(path_trans.read_text(encoding="utf-8"))

# Index par nom de fichier
trans_by_name = {t.get("file_name"): t for t in trans}

file_names = meta.get("File name", [])
n = len(file_names)

# toutes les clés présentes dans transcriptions (sauf file_name)
trans_keys = set()
for t in trans:
    for k in t.keys():
        if k != "file_name":
            trans_keys.add(k)

# colonnes cibles (prefixées pour éviter collision)
col_map = {k: f"trans_{k}" for k in sorted(trans_keys)}

# initialise les colonnes
for col in col_map.values():
    if col not in meta:
        meta[col] = [""] * n

# remplissage
for i, fname in enumerate(file_names):
    t = trans_by_name.get(fname)
    if not t:
        continue
    for src_key, dst_key in col_map.items():
        meta[dst_key][i] = t.get(src_key, "")

out_path.write_text(json.dumps(meta, ensure_ascii=False, indent=2), encoding="utf-8")
print("ok ->", out_path)


ok -> data/test20/metadata_test20_enrichi.json


In [16]:
import json
import csv
from pathlib import Path

in_path = Path("data/test20/metadata_test20_enrichi.json")
out_path = Path("data/test20/metadata_test20_enrichi.csv")

data = json.loads(in_path.read_text(encoding="utf-8"))

fields = list(data.keys())
n = len(data.get("File name", []))

with out_path.open("w", newline="", encoding="utf-8") as f:
    writer = csv.writer(f)
    writer.writerow(fields)
    for i in range(n):
        row = []
        for k in fields:
            v = data.get(k, "")
            if isinstance(v, list):
                row.append(v[i] if i < len(v) else "")
            else:
                row.append(v)
        writer.writerow(row)

print("ok ->", out_path)


ok -> data/test20/metadata_test20_enrichi.csv


In [17]:
import json
from pathlib import Path

in_path = Path("data/metadata_monu.json")
out_path = Path("data/metadata_monu_52.json")

data = json.loads(in_path.read_text(encoding="utf-8"))

# longueur basée sur File name
n = min(52, len(data.get("File name", [])))

# supprimer colonnes
data.pop("longitude", None)
data.pop("latitude", None)

# tronquer toutes les colonnes listes
for k, v in data.items():
    if isinstance(v, list):
        data[k] = v[:n]

out_path.write_text(json.dumps(data, ensure_ascii=False, indent=2), encoding="utf-8")
print("ok ->", out_path)


ok -> data/metadata_monu_52.json


In [18]:
import json
import csv
from pathlib import Path

in_path = Path("data/metadata_monu_52.json")
out_path = Path("data/metadata_monu_52.csv")

data = json.loads(in_path.read_text(encoding="utf-8"))

fields = list(data.keys())
n = len(data.get("File name", []))

with out_path.open("w", newline="", encoding="utf-8") as f:
    writer = csv.writer(f)
    writer.writerow(fields)
    for i in range(n):
        row = []
        for k in fields:
            v = data.get(k, "")
            if isinstance(v, list):
                row.append(v[i] if i < len(v) else "")
            else:
                row.append(v)
        writer.writerow(row)

print("ok ->", out_path)


ok -> data/metadata_monu_52.csv


In [19]:
import json
import csv
from pathlib import Path

path_json = Path("data/metadata.json")
path_csv = Path("data/metadata.csv")

remove_cols = {
    "City", "Raw text", "Hamlet", "Monument", "Other", "coordonnées"
}
rename_from = "code_insee"
rename_to = "code_insee_commune"

# --- JSON ---
data = json.loads(path_json.read_text(encoding="utf-8"))

# remove columns
for col in remove_cols:
    data.pop(col, None)

# rename column
if rename_from in data:
    data[rename_to] = data.pop(rename_from)

path_json.write_text(json.dumps(data, ensure_ascii=False, indent=2), encoding="utf-8")

# --- CSV ---
with path_csv.open("r", encoding="utf-8") as f_in:
    reader = csv.DictReader(f_in)
    fieldnames = []
    for fn in reader.fieldnames:
        if fn in remove_cols:
            continue
        if fn == rename_from:
            fieldnames.append(rename_to)
        else:
            fieldnames.append(fn)

    rows = []
    for row in reader:
        new_row = {}
        for k, v in row.items():
            if k in remove_cols:
                continue
            if k == rename_from:
                new_row[rename_to] = v
            else:
                new_row[k] = v
        rows.append(new_row)

with path_csv.open("w", newline="", encoding="utf-8") as f_out:
    writer = csv.DictWriter(f_out, fieldnames=fieldnames)
    writer.writeheader()
    writer.writerows(rows)

print("ok ->", path_json, "and", path_csv)


ok -> data/metadata.json and data/metadata.csv


In [21]:
!pip install PIL

ERROR: Could not find a version that satisfies the requirement PIL (from versions: none)
ERROR: No matching distribution found for PIL


In [22]:
import torch

ModuleNotFoundError: No module named 'torch'

In [23]:
!pip install torch

  Using cached torch-2.9.1-cp313-none-macosx_11_0_arm64.whl.metadata (30 kB)
  Using cached sympy-1.14.0-py3-none-any.whl.metadata (12 kB)
  Using cached fsspec-2025.12.0-py3-none-any.whl.metadata (10 kB)
  Using cached mpmath-1.3.0-py3-none-any.whl.metadata (8.6 kB)
Using cached torch-2.9.1-cp313-none-macosx_11_0_arm64.whl (74.5 MB)
Using cached fsspec-2025.12.0-py3-none-any.whl (201 kB)
Using cached sympy-1.14.0-py3-none-any.whl (6.3 MB)
Using cached mpmath-1.3.0-py3-none-any.whl (536 kB)
   ━━━━━━━━╺━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1/5 [sympy]^C
